# <font color='blue'>Uni-Facef - Pyspark - Atividade</font>

**Trabalho Python Facef - André Borges [22290**]

Essa atividade tem como objetivo descobrir um pouco mais sobre o cliente e traçar um perfil do mesmo. Visto que temos as informações pessoais do cliemte em um arquivo chamado "cliente_fake.csv", e os dados transacionais desse cliente no arquivo chamado "venda_fake". E idéia do exercicio é a manipulação dos dados de forma a fazer agregações, join nos dados, podendo ser necessário o uso de UDFs, gerando um Dataframe Final:

#### O Dataframe final deve estar agrupado por cliente (cliente_id), e conter as seguintes informações:

- cliente_id - O identificador do cliente
- idade - Calcular a idade do cliente (valor inteiro)
- recencia - Calcular a quantos meses faz que o cliente não compra (valor inteiro)
- qt_total_compras - Quantidade total de compras (pedidos distintos)
- vr_total_compras = Valor total de compras do cliente
- qt_total_itens - Quantidade total de itens comprados pelo cliente (produto_id distintos)
- qt_max_parcelas - Quantidade máxima de parcelas que o cliente já utilizou
- perfil_cliente - Deve mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

OBS: O dataset "venda_fake" está a nível de itens. Pode haver mais de um item por pedido, ito é, o numero do pedido pode estar duplicado nodataset. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 66kB/s 
     |████████████████████████████████| 204kB 39.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=0a3cb7e012eee5e78ae6cd161a05cc6ed9289f348a970f7913e79a715eb3d9ce
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
# coding: utf-8
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName('ConhecendoCliente') \
    .getOrCreate()

#### Dataset de cliente

In [4]:
df_cliente = spark.read \
    .option("delimiter", "|") \
    .csv('cliente_fake.csv', header=True)

df_cliente.orderBy('cliente_id').show()

+--------------+----------+-------------+
|          nome|cliente_id|dt_nascimento|
+--------------+----------+-------------+
|Cliente 100001|    100001|   2009-04-16|
|Cliente 100002|    100002|   1974-08-29|
|Cliente 100003|    100003|   1981-12-05|
|Cliente 100004|    100004|   1968-08-08|
|Cliente 100005|    100005|   1982-11-29|
|Cliente 100006|    100006|   1948-04-01|
|Cliente 100007|    100007|   1976-11-04|
|Cliente 100008|    100008|   1980-12-01|
|Cliente 100009|    100009|   1981-03-05|
|Cliente 100010|    100010|   1984-03-20|
|Cliente 100011|    100011|   1977-11-22|
|Cliente 100012|    100012|   1974-02-04|
|Cliente 100013|    100013|   1964-06-21|
|Cliente 100014|    100014|   1964-05-24|
|Cliente 100015|    100015|   1946-10-18|
|Cliente 100016|    100016|   1972-03-24|
|Cliente 100017|    100017|   1982-01-19|
|Cliente 100018|    100018|   1983-04-26|
|Cliente 100019|    100019|   1971-09-09|
|Cliente 100020|    100020|   1989-06-18|
+--------------+----------+-------

#### Dataset de pedido

In [5]:
df_venda = spark.read \
    .option("delimiter", "|") \
    .csv('venda_fake.csv', header=True)

df_venda.orderBy('cliente_id').show(df_venda.count(), False)

+--------------+----------+------------------------+----------+-----------+--------------+------------+-------+--------+
|pedido_id     |cliente_id|dt_pedido               |produto_id|qt_parcelas|forma_pagto   |vr_unit_item|qt_item|tp_canal|
+--------------+----------+------------------------+----------+-----------+--------------+------------+-------+--------+
|10000120200626|100001    |2020-06-26T00:00:00.000Z|7560984   |16         |cartao credito|1399.04     |1      |loja    |
|10000220200221|100002    |2020-02-21T00:00:00.000Z|7637880   |9          |carne         |1280.07     |1      |loja    |
|10000320200730|100003    |2020-07-30T00:00:00.000Z|7642944   |10         |outros        |998.96      |1      |loja    |
|10000320200810|100003    |2020-08-10T00:00:00.000Z|7642956   |9          |outros        |899         |1      |loja    |
|10000320200810|100003    |2020-08-10T00:00:00.000Z|6906072   |9          |outros        |6.99        |1      |loja    |
|10000420200107|100004    |2020-

 #### Dataframe Final


In [78]:
import pyspark.sql.functions as spark

cliente = df_cliente.select(spark.col('cliente_id'))

idade = df_cliente.select(spark.col('cliente_id'), (spark.year(spark.current_date()) - spark.year('dt_nascimento')).alias('idade'))                                                     

recencia = df_venda \
  .select(spark.col('cliente_id'), spark.col('dt_pedido').cast('timestamp')) \
  .withColumn('recencia', spark.round(spark.months_between(spark.current_date(), 'dt_pedido').cast("integer"))) \
  .groupBy('cliente_id') \
  .max('recencia') \
  .select(spark.col('cliente_id'), spark.col('max(recencia)').alias("recencia"))

qt_total_compras = df_venda \
  .select(spark.col('cliente_id'), spark.col('pedido_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .select(spark.col('cliente_id'), spark.col('count').alias("qt_total_compras")) 

vr_total_compras = df_venda \
  .select(
      spark.col('cliente_id'),
      spark.col('vr_unit_item').cast('numeric(10,2)'),
      spark.col('qt_item').cast('integer')) \
  .withColumn('vr_total_items', 
              spark.col('vr_unit_item').cast('numeric(10,2)') * \
              spark.col('qt_item').cast('integer')) \
  .groupBy('cliente_id') \
  .sum('vr_total_items') \
  .select(spark.col('cliente_id'), spark.col('sum(vr_total_items)').alias("vr_total_compras")) 

qt_total_itens = df_venda \
  .select(spark.col('cliente_id'), spark.col('produto_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .select(spark.col('cliente_id'), spark.col('count').alias("qt_total_itens"))   

qt_max_parcelas = df_venda \
  .select(spark.col('cliente_id'), spark.col('qt_parcelas').cast('integer')) \
  .distinct() \
  .groupBy('cliente_id') \
  .max('qt_parcelas') \
  .select(spark.col('cliente_id'), spark.col('max(qt_parcelas)').alias("qt_max_parcelas"))  

cliente_tpc = df_venda \
  .select(spark.col('tp_canal'), spark.col('cliente_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .agg(spark.max("tp_canal").alias("tp_canal"))

cliente_qtc = df_venda \
  .select(spark.col('tp_canal'), spark.col('cliente_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .select(spark.col('cliente_id'), spark.col('count').alias("qt_canal"))     
  
perfil_cliente = cliente_tpc.join(cliente_qtc, 'cliente_id') \
  .withColumn('perfil_cliente', spark.when(spark.col('qt_canal') == 1, spark.col('tp_canal'))
  .otherwise("multicanal"))

perfil_cliente = perfil_cliente.select(spark.col('cliente_id'),spark.col('perfil_cliente'))

output = cliente.join(idade, 'cliente_id') \
  .join(recencia, 'cliente_id') \
  .join(qt_total_compras, 'cliente_id') \
  .join(vr_total_compras, 'cliente_id') \
  .join(qt_total_itens, 'cliente_id') \
  .join(qt_max_parcelas, 'cliente_id') \
  .join(perfil_cliente, 'cliente_id') 

output.orderBy('cliente_id').show()

+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|cliente_id|idade|recencia|qt_total_compras|vr_total_compras|qt_total_itens|qt_max_parcelas|perfil_cliente|
+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|    100001|   11|       4|               1|         1399.04|             1|             16|          loja|
|    100002|   46|       8|               1|         1280.07|             1|              9|          loja|
|    100003|   39|       3|               2|         1904.95|             3|             10|          loja|
|    100004|   52|       9|               2|         6860.08|             2|             24|    multicanal|
|    100005|   38|       9|               2|         3207.28|             2|             15|          loja|
|    100006|   72|       2|               1|         3170.72|             3|              9|          loja|
|    100007|   44|       8| 